# กลไก Attention และ Transformer

ข้อเสียสำคัญของเครือข่ายแบบ Recurrent คือทุกคำในลำดับมีผลกระทบต่อผลลัพธ์เท่ากัน ซึ่งทำให้ประสิทธิภาพของโมเดล LSTM แบบ encoder-decoder มาตรฐานสำหรับงานลำดับต่อลำดับ เช่น การระบุชื่อเฉพาะ (Named Entity Recognition) และการแปลภาษา (Machine Translation) ไม่ดีเท่าที่ควร ในความเป็นจริง คำบางคำในลำดับข้อมูลเข้าอาจมีผลกระทบต่อผลลัพธ์มากกว่าคำอื่นๆ

ลองพิจารณาโมเดลลำดับต่อลำดับ เช่น การแปลภาษา โมเดลนี้ถูกสร้างขึ้นโดยใช้เครือข่ายแบบ Recurrent สองตัว โดยเครือข่ายตัวหนึ่ง (**encoder**) จะบีบอัดลำดับข้อมูลเข้าไปเป็นสถานะซ่อน และอีกตัวหนึ่ง (**decoder**) จะคลายสถานะซ่อนนี้ออกมาเป็นผลลัพธ์ที่แปลแล้ว ปัญหาของวิธีนี้คือสถานะสุดท้ายของเครือข่ายจะมีปัญหาในการจดจำจุดเริ่มต้นของประโยค ซึ่งส่งผลให้คุณภาพของโมเดลลดลงเมื่อประโยคยาวขึ้น

**กลไก Attention** เป็นวิธีการที่ช่วยให้น้ำหนักความสำคัญของแต่ละเวกเตอร์ข้อมูลเข้ามีผลต่อการทำนายผลลัพธ์ของ RNN โดยวิธีการนี้จะสร้างทางลัดระหว่างสถานะกลางของ RNN ข้อมูลเข้าและ RNN ข้อมูลออก ในลักษณะนี้ เมื่อสร้างสัญลักษณ์ผลลัพธ์ $y_t$ เราจะพิจารณาสถานะซ่อนของข้อมูลเข้าทั้งหมด $h_i$ โดยมีค่าสัมประสิทธิ์น้ำหนักที่แตกต่างกัน $\alpha_{t,i}$

![ภาพแสดงโมเดล encoder/decoder พร้อมชั้น Attention แบบ additive](../../../../../translated_images/th/encoder-decoder-attention.7a726296894fb567.webp)
*โมเดล encoder-decoder พร้อมกลไก Attention แบบ additive จาก [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) อ้างอิงจาก [บล็อกโพสต์นี้](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

เมทริกซ์ Attention $\{\alpha_{i,j}\}$ จะเป็นตัวแทนระดับที่คำบางคำในข้อมูลเข้ามีบทบาทในการสร้างคำในลำดับผลลัพธ์ ตัวอย่างของเมทริกซ์ดังกล่าวแสดงอยู่ด้านล่าง:

![ภาพแสดงตัวอย่างการจัดแนวที่พบโดย RNNsearch-50 จาก Bahdanau - arviz.org](../../../../../translated_images/th/bahdanau-fig3.09ba2d37f202a6af.webp)

*ภาพจาก [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

กลไก Attention มีบทบาทสำคัญในความก้าวหน้าของงานประมวลผลภาษาธรรมชาติในปัจจุบันหรือใกล้เคียงปัจจุบัน อย่างไรก็ตาม การเพิ่ม Attention ทำให้จำนวนพารามิเตอร์ของโมเดลเพิ่มขึ้นอย่างมาก ซึ่งนำไปสู่ปัญหาการปรับขนาดของ RNN ข้อจำกัดสำคัญของการปรับขนาด RNN คือธรรมชาติของการทำงานแบบ Recurrent ทำให้การประมวลผลแบบ batch และการขนานเป็นเรื่องท้าทาย ใน RNN แต่ละองค์ประกอบของลำดับต้องถูกประมวลผลตามลำดับ ซึ่งหมายความว่าไม่สามารถขนานได้ง่าย

การนำกลไก Attention มาใช้ร่วมกับข้อจำกัดนี้นำไปสู่การสร้างโมเดล Transformer ซึ่งเป็น State of the Art ในปัจจุบันที่เราใช้กัน เช่น BERT และ OpenGPT3

## โมเดล Transformer

แทนที่จะส่งต่อบริบทของการทำนายครั้งก่อนเข้าสู่ขั้นตอนการประเมินผลครั้งถัดไป **โมเดล Transformer** ใช้ **positional encodings** และ **attention** เพื่อจับบริบทของข้อมูลเข้าภายในหน้าต่างข้อความที่กำหนด ภาพด้านล่างแสดงวิธีที่ positional encodings ร่วมกับ attention สามารถจับบริบทภายในหน้าต่างที่กำหนดได้

![GIF แสดงวิธีการประเมินผลในโมเดล Transformer](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

เนื่องจากตำแหน่งข้อมูลเข้าทุกตำแหน่งถูกแมปไปยังตำแหน่งข้อมูลออกอย่างอิสระ โมเดล Transformer สามารถขนานได้ดีกว่า RNN ซึ่งช่วยให้สร้างโมเดลภาษาที่ใหญ่ขึ้นและแสดงออกได้มากขึ้น หัว Attention แต่ละหัวสามารถใช้เรียนรู้ความสัมพันธ์ระหว่างคำที่แตกต่างกัน ซึ่งช่วยปรับปรุงงานประมวลผลภาษาธรรมชาติในขั้นตอนถัดไป

## การสร้างโมเดล Transformer แบบง่าย

Keras ไม่มีชั้น Transformer ที่สร้างไว้แล้ว แต่เราสามารถสร้างขึ้นเองได้ เช่นเคย เราจะมุ่งเน้นไปที่การจัดประเภทข้อความในชุดข้อมูล AG News แต่ควรกล่าวว่าโมเดล Transformer แสดงผลลัพธ์ที่ดีที่สุดในงาน NLP ที่ยากกว่า


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

เลเยอร์ใหม่ใน Keras ควรสืบทอดคลาส `Layer` และต้องมีการใช้งานเมธอด `call` เริ่มต้นด้วยเลเยอร์ **Positional Embedding** เราจะใช้ [โค้ดบางส่วนจากเอกสาร Keras อย่างเป็นทางการ](https://keras.io/examples/nlp/text_classification_with_transformer/) โดยเราจะสมมติว่าเราเติมข้อมูลในลำดับอินพุตทั้งหมดให้มีความยาว `maxlen`


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

เลเยอร์นี้ประกอบด้วย `Embedding` สองเลเยอร์: สำหรับฝังตัวอักษร (ในลักษณะที่เราได้พูดถึงก่อนหน้านี้) และตำแหน่งของตัวอักษร ตำแหน่งของตัวอักษรจะถูกสร้างขึ้นเป็นลำดับของตัวเลขธรรมชาติจาก 0 ถึง `maxlen` โดยใช้ `tf.range` และจากนั้นจะถูกส่งผ่านเลเยอร์ฝังตัวอักษร เวกเตอร์ฝังตัวที่ได้ทั้งสองตัวจะถูกนำมาบวกกัน เพื่อสร้างตัวแทนที่ฝังตำแหน่งของอินพุตในรูปแบบ `maxlen`$\times$`embed_dim`

ตอนนี้ เรามาเริ่มต้นสร้างบล็อกของ Transformer กัน บล็อกนี้จะรับผลลัพธ์จากเลเยอร์ฝังตัวที่เราได้กำหนดไว้ก่อนหน้านี้:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

ตอนนี้เราพร้อมที่จะกำหนดโมเดล Transformer แบบสมบูรณ์แล้ว:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## โมเดล BERT Transformer

**BERT** (Bidirectional Encoder Representations from Transformers) เป็นเครือข่ายทรานส์ฟอร์เมอร์ขนาดใหญ่มากที่มีหลายชั้น โดย *BERT-base* มี 12 ชั้น และ *BERT-large* มี 24 ชั้น โมเดลนี้ถูกฝึกเบื้องต้นด้วยชุดข้อมูลข้อความขนาดใหญ่ (WikiPedia + หนังสือ) โดยใช้การฝึกแบบไม่มีการกำกับดูแล (การทำนายคำที่ถูกปิดบังในประโยค) ในระหว่างการฝึกเบื้องต้น โมเดลจะเรียนรู้ความเข้าใจในภาษาระดับสูง ซึ่งสามารถนำไปใช้กับชุดข้อมูลอื่น ๆ ได้โดยการปรับแต่งเพิ่มเติม กระบวนการนี้เรียกว่า **การเรียนรู้แบบถ่ายโอน** 

![ภาพจาก http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/th/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

มีสถาปัตยกรรม Transformer หลากหลายรูปแบบ เช่น BERT, DistilBERT, BigBird, OpenGPT3 และอื่น ๆ ที่สามารถปรับแต่งเพิ่มเติมได้ 

มาดูกันว่าเราจะใช้โมเดล BERT ที่ฝึกเบื้องต้นแล้วในการแก้ปัญหาการจัดประเภทลำดับแบบดั้งเดิมของเราได้อย่างไร เราจะยืมแนวคิดและโค้ดบางส่วนจาก [เอกสารประกอบอย่างเป็นทางการ](https://www.tensorflow.org/text/tutorials/classify_text_with_bert)

ในการโหลดโมเดลที่ฝึกเบื้องต้นแล้ว เราจะใช้ **Tensorflow hub** ก่อนอื่น มาลองโหลดตัวแปลงเวกเตอร์เฉพาะของ BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

สิ่งสำคัญคือคุณต้องใช้ตัวเวกเตอร์ที่เหมือนกับตัวที่เครือข่ายต้นฉบับถูกฝึกมา นอกจากนี้ ตัวเวกเตอร์ BERT จะคืนค่ามาเป็นสามองค์ประกอบ:
* `input_word_ids` ซึ่งเป็นลำดับของหมายเลขโทเค็นสำหรับประโยคที่ป้อนเข้า
* `input_mask` แสดงว่าช่วงใดของลำดับมีข้อมูลที่ป้อนเข้าจริง และช่วงใดเป็นการเติมข้อมูล (padding) ซึ่งคล้ายกับมาสก์ที่สร้างโดยเลเยอร์ `Masking`
* `input_type_ids` ใช้สำหรับงานการสร้างแบบจำลองภาษา และช่วยให้สามารถระบุสองประโยคในลำดับเดียวกันได้

จากนั้น เราสามารถสร้างตัวดึงคุณลักษณะ BERT ได้:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


ดังนั้น BERT layer จะคืนค่าผลลัพธ์ที่มีประโยชน์หลายอย่าง:
* `pooled_output` เป็นผลลัพธ์จากการเฉลี่ยค่าของทุก token ใน sequence คุณสามารถมองว่ามันเป็นการฝังความหมายเชิงอัจฉริยะของเครือข่ายทั้งหมด ซึ่งเทียบเท่ากับผลลัพธ์ของ `GlobalAveragePooling1D` layer ในโมเดลก่อนหน้าของเรา
* `sequence_output` เป็นผลลัพธ์ของ transformer layer สุดท้าย (สอดคล้องกับผลลัพธ์ของ `TransformerBlock` ในโมเดลด้านบนของเรา)
* `encoder_outputs` เป็นผลลัพธ์ของ transformer layers ทั้งหมด เนื่องจากเราได้โหลดโมเดล BERT ที่มี 4-layer (ซึ่งคุณอาจเดาได้จากชื่อที่มี `4_H`) มันจึงมี 4 tensors โดย tensor สุดท้ายจะเหมือนกับ `sequence_output`

ตอนนี้เราจะกำหนดโมเดลการจัดประเภทแบบ end-to-end เราจะใช้ *การกำหนดโมเดลแบบฟังก์ชัน* โดยเราจะกำหนด input ของโมเดล และจากนั้นให้ชุดของนิพจน์เพื่อคำนวณ output ของมัน นอกจากนี้เราจะทำให้ weights ของโมเดล BERT ไม่สามารถ train ได้ และ train เฉพาะตัว classifier สุดท้าย:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


แม้ว่าจะมีพารามิเตอร์ที่สามารถฝึกได้เพียงไม่กี่ตัว แต่กระบวนการก็ยังค่อนข้างช้า เพราะตัวดึงคุณลักษณะของ BERT นั้นใช้ทรัพยากรการคำนวณสูง ดูเหมือนว่าเราจะไม่สามารถบรรลุความแม่นยำที่เหมาะสมได้ อาจเป็นเพราะการฝึกที่ไม่เพียงพอ หรือพารามิเตอร์ของโมเดลที่ไม่เพียงพอ

ลองปลดล็อกน้ำหนักของ BERT และฝึกมันด้วยกันดู การทำเช่นนี้ต้องใช้ค่า learning rate ที่เล็กมาก และกลยุทธ์การฝึกที่ระมัดระวังมากขึ้นด้วย **warmup** โดยใช้ตัวปรับแต่ง **AdamW** เราจะใช้แพ็กเกจ `tf-models-official` เพื่อสร้างตัวปรับแต่ง:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

การฝึกอบรมดำเนินไปค่อนข้างช้า - แต่คุณอาจต้องการทดลองและฝึกโมเดลเป็นระยะเวลาไม่กี่รอบ (5-10) และดูว่าคุณสามารถได้ผลลัพธ์ที่ดีที่สุดเมื่อเปรียบเทียบกับวิธีที่เราเคยใช้มาก่อนหรือไม่

## Huggingface Transformers Library

อีกวิธีหนึ่งที่เป็นที่นิยม (และง่ายกว่าเล็กน้อย) ในการใช้งานโมเดล Transformer คือ [HuggingFace package](https://github.com/huggingface/), ซึ่งให้โครงสร้างพื้นฐานที่ง่ายสำหรับงาน NLP ต่างๆ โดยสามารถใช้งานได้ทั้งใน Tensorflow และ PyTorch ซึ่งเป็นเฟรมเวิร์กเครือข่ายประสาทเทียมที่ได้รับความนิยมอีกตัวหนึ่ง

> **Note**: หากคุณไม่สนใจที่จะดูวิธีการทำงานของ Transformers library - คุณสามารถข้ามไปยังส่วนท้ายของโน้ตบุ๊กนี้ได้ เพราะคุณจะไม่เห็นสิ่งที่แตกต่างอย่างมีนัยสำคัญจากสิ่งที่เราได้ทำไปก่อนหน้านี้ เราจะทำขั้นตอนเดิมในการฝึกโมเดล BERT โดยใช้ไลบรารีที่แตกต่างกันและโมเดลที่ใหญ่ขึ้นอย่างมาก ดังนั้นกระบวนการนี้จะเกี่ยวข้องกับการฝึกอบรมที่ค่อนข้างยาวนาน คุณอาจต้องการเพียงแค่ดูโค้ดผ่านๆ

มาดูกันว่าปัญหาของเราสามารถแก้ไขได้อย่างไรโดยใช้ [Huggingface Transformers](http://huggingface.co)


สิ่งแรกที่เราต้องทำคือเลือกโมเดลที่เราจะใช้ นอกจากโมเดลที่มีอยู่ในตัวแล้ว Huggingface ยังมี [คลังโมเดลออนไลน์](https://huggingface.co/models) ซึ่งคุณสามารถค้นหาโมเดลที่ผ่านการฝึกฝนมาแล้วจากชุมชนได้อีกมากมาย โมเดลเหล่านี้สามารถโหลดและใช้งานได้เพียงแค่ระบุชื่อโมเดล ไฟล์ไบนารีที่จำเป็นทั้งหมดสำหรับโมเดลจะถูกดาวน์โหลดโดยอัตโนมัติ

ในบางครั้งคุณอาจต้องโหลดโมเดลของคุณเอง ซึ่งในกรณีนี้คุณสามารถระบุไดเรกทอรีที่มีไฟล์ที่เกี่ยวข้องทั้งหมด รวมถึงพารามิเตอร์สำหรับ tokenizer ไฟล์ `config.json` ที่มีพารามิเตอร์ของโมเดล น้ำหนักไบนารี เป็นต้น

จากชื่อโมเดล เราสามารถสร้างทั้งโมเดลและ tokenizer ได้ ลองเริ่มต้นด้วย tokenizer กัน:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

วัตถุ `tokenizer` มีฟังก์ชัน `encode` ที่สามารถใช้เข้ารหัสข้อความได้โดยตรง:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

เรายังสามารถใช้ตัวแยกคำเพื่อเข้ารหัสลำดับในรูปแบบที่เหมาะสมสำหรับการส่งผ่านไปยังโมเดล เช่น รวมถึงฟิลด์ `token_ids`, `input_mask` เป็นต้น นอกจากนี้ เรายังสามารถระบุได้ว่าเราต้องการ Tensorflow tensors โดยการให้ค่าอาร์กิวเมนต์ `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

ในกรณีนี้ เราจะใช้โมเดล BERT ที่ถูกฝึกมาแล้วชื่อ `bert-base-uncased` คำว่า *Uncased* หมายความว่าโมเดลนี้ไม่แยกแยะตัวพิมพ์เล็กและตัวพิมพ์ใหญ่

เมื่อเราฝึกโมเดล เราจำเป็นต้องให้ลำดับที่ถูกแปลงเป็นโทเค็นเป็นอินพุต ดังนั้นเราจะออกแบบกระบวนการประมวลผลข้อมูล เนื่องจาก `tokenizer.encode` เป็นฟังก์ชันใน Python เราจะใช้วิธีเดียวกับในหน่วยก่อนหน้าโดยเรียกใช้ผ่าน `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

ตอนนี้เราสามารถโหลดโมเดลจริงโดยใช้แพ็กเกจ `BertForSequenceClassification` ซึ่งจะช่วยให้โมเดลของเรามีสถาปัตยกรรมที่จำเป็นสำหรับการจำแนกประเภท รวมถึงตัวจำแนกขั้นสุดท้าย คุณจะเห็นข้อความเตือนที่ระบุว่าน้ำหนักของตัวจำแนกขั้นสุดท้ายยังไม่ได้รับการเริ่มต้น และโมเดลจะต้องการการฝึกอบรมล่วงหน้า - ซึ่งเป็นเรื่องปกติอย่างสมบูรณ์ เพราะนั่นคือสิ่งที่เรากำลังจะทำ!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


ดังที่คุณเห็นจาก `summary()` โมเดลมีพารามิเตอร์เกือบ 110 ล้านตัว! โดยปกติแล้ว หากเราต้องการงานการจำแนกประเภทที่ง่ายบนชุดข้อมูลที่ค่อนข้างเล็ก เราไม่ต้องการฝึกชั้นฐานของ BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


ตอนนี้เราพร้อมที่จะเริ่มการฝึกแล้ว!

> **Note**: การฝึกโมเดล BERT แบบเต็มรูปแบบอาจใช้เวลานานมาก! ดังนั้นเราจะฝึกเพียง 32 ชุดข้อมูลแรกเท่านั้น นี่เป็นเพียงการแสดงให้เห็นว่าการตั้งค่าการฝึกโมเดลเป็นอย่างไร หากคุณสนใจที่จะลองฝึกแบบเต็มรูปแบบ - เพียงแค่ลบพารามิเตอร์ `steps_per_epoch` และ `validation_steps` ออก และเตรียมตัวรอ!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


หากคุณเพิ่มจำนวนรอบการทำงานและรอให้นานพอ รวมถึงฝึกอบรมหลายรอบ คุณสามารถคาดหวังได้ว่า BERT classification จะให้ความแม่นยำที่ดีที่สุด! นั่นเป็นเพราะว่า BERT เข้าใจโครงสร้างของภาษาได้ดีอยู่แล้ว และเราจำเป็นต้องปรับแต่งตัวจัดประเภทขั้นสุดท้ายเท่านั้น อย่างไรก็ตาม เนื่องจาก BERT เป็นโมเดลขนาดใหญ่ กระบวนการฝึกอบรมทั้งหมดจึงใช้เวลานาน และต้องการพลังการประมวลผลที่สูงมาก! (GPU และควรมีมากกว่าหนึ่งตัว)

> **Note:** ในตัวอย่างของเรา เราได้ใช้หนึ่งในโมเดล BERT ที่ผ่านการฝึกอบรมล่วงหน้าขนาดเล็กที่สุด ยังมีโมเดลที่ใหญ่กว่าซึ่งมีแนวโน้มที่จะให้ผลลัพธ์ที่ดีกว่า


## สาระสำคัญ

ในหน่วยนี้ เราได้เรียนรู้เกี่ยวกับสถาปัตยกรรมโมเดลที่ทันสมัยซึ่งอิงกับ **transformers** เราได้นำมาใช้กับงานการจัดประเภทข้อความของเรา แต่ในทำนองเดียวกัน โมเดล BERT ยังสามารถนำไปใช้กับงานอื่น ๆ ใน NLP เช่น การดึงข้อมูลเอนทิตี การตอบคำถาม และงานอื่น ๆ ได้อีกด้วย

โมเดล Transformer ถือเป็นเทคโนโลยีที่ล้ำหน้าที่สุดในปัจจุบันสำหรับ NLP และในหลาย ๆ กรณี ควรเป็นตัวเลือกแรกที่คุณเริ่มทดลองใช้เมื่อพัฒนาวิธีแก้ปัญหา NLP แบบกำหนดเอง อย่างไรก็ตาม การทำความเข้าใจหลักการพื้นฐานของ recurrent neural networks ที่ได้กล่าวถึงในโมดูลนี้ก็มีความสำคัญอย่างยิ่ง หากคุณต้องการสร้างโมเดลประสาทขั้นสูง



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามนุษย์มืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
